In [29]:
import numpy as np
import cv2
import os
import csv
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [3]:
# TODO: See if want to reduce image size or perform PCA
def data_preprocessing(negative_pics, positive_pics):
    # Class 0 => No cracks 
    # Class 1 => Cracks 

    # Before reshape => (20000, 227, 227)
    # Want => (20000, 51529, 1)
    negative_images = []

    # Iterate over the folder with no cracks  
    for picture in os.scandir(negative_pics):
        if picture.is_file():
            pic_path = negative_pics + '/' + picture.name
            image = cv2.imread(pic_path)
            # Remove 3 channels 
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Each image is (227, 227, 3)
            # Grayscale => (227, 227, 1)
            data = np.asarray(gray_image)
            data = data.reshape((227*227))
            # Add extra last column to help with labeling
            data = np.append(data, 0)
            negative_images.append(data)

    print("Shape of negative images", np.array(negative_images).shape)
    positive_images = []

    # Iterative over the folder with cracks
    for picture in os.scandir(positive_pics):
        if picture.is_file():
            pic_path = positive_pics + '/' + picture.name
            image = cv2.imread(pic_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            data = np.asarray(gray_image)
            data = data.reshape((227*227))
            data = np.append(data, 1)
            positive_images.append(data)
        
    print("Shape of positive images", np.array(positive_images).shape)
    #print(positive_images)

    all_images = np.concatenate((negative_images, positive_images))

    # TODO: Seed the shuffle??
    np.random.shuffle(all_images)
    # 75 and 25 split (30000) and (10000)
    x_train, x_test = all_images[:30000], all_images[30000:]

    y_train = []
    y_test = []

    for point in x_train:
        if (point[-1] == 0):
            # OHE for class 0
            y_train.append([1, 0])
        if (point[-1] == 1):
            # OHE for class 1
            y_train.append([0, 1])

    # TODO: See if this needs to be done before reshaping!!!
    # drop the label point 
    x_train = np.delete(x_train, obj=-1, axis=1)
    x_train = x_train.astype('float32')
    # Normalize the data
    x_train /= 255
    #x_train = x_train[:, :, np.newaxis]

    for point in x_test:
        if (point[-1] == 0):
            y_test.append([1, 0])
        if (point[-1] == 1):
            y_test.append([0, 1])
        
    # TODO: Add this to the 1 Hidden Layer Cell
    # TODO: Understand the formula in the hw5 doc
    # drop the label point 
    x_test = np.delete(x_test, obj=-1, axis=1)
    x_test = x_test.astype('float32')
    x_test /= 255
    #x_test = x_test[:, :, np.newaxis]

    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    x_test = np.asarray(x_test)
    y_test = np.asarray(y_test)

    return (x_train, y_train), (x_test, y_test)

In [4]:
negative_pics = 'ConcreteCracks/Negative'
positive_pics = 'ConcreteCracks/Positive'

(x_train, y_train), (x_test, y_test) = data_preprocessing(negative_pics, positive_pics)

print("Shape of x_train", x_train.shape)
print("Shape of y_train", y_train.shape)
print("Shape of x_test", x_test.shape)
print("Shape of y_test", y_test.shape)

Shape of negative images (20000, 51530)
Shape of positive images (20000, 51530)
Shape of x_train (30000, 51529)
Shape of y_train (30000, 2)
Shape of x_test (10000, 51529)
Shape of y_test (10000, 2)


In [5]:
kmeans = KMeans(n_clusters=2, random_state=1).fit(x_train)

In [33]:
def transformLabel(label):
    k_means_label = []

    for i in range(len(label)):
        if (label[i][0] == 1):
            k_means_label.append(0)
        else:
            k_means_label.append(1)

    print("New labels arrangement: ", np.array(k_means_label).shape)

    return k_means_label

k_means_y_train = transformLabel(y_train)
k_means_y_test = transformLabel(y_test)

New labels arrangement:  (30000,)
New labels arrangement:  (10000,)


In [13]:
def extractLabelInfo(predict, label):
    reference_labels = {}
    label = np.array(label)

    for i in range(2):
        indices = np.where(predict == i)
        actual_values = []
        for j in range(len(indices)):
            temp = label[(indices[0][j])]
            actual_values.append(temp)

        # Get the class that was actually assigned to the cluster
        cluster_label = max(set(actual_values), key=actual_values.count)

        reference_labels.update({i:cluster_label})
    
    return reference_labels

In [37]:
def computeAccuracy(prediction, label):
    reference_labels = extractLabelInfo(prediction, label)
    print(reference_labels)

    number_labels = []
    for i in range(len(prediction)):
        number_labels.append(reference_labels[prediction[i]])

    accuracy = accuracy_score(number_labels, label)*100

    return accuracy

training_accuracy = computeAccuracy(kmeans.labels_, k_means_y_train)
print("Training Accuracy: ", training_accuracy)

prediction = kmeans.predict(x_test)

testing_accuracy = computeAccuracy(prediction, k_means_y_test)
print("Testing Accuracy: ", testing_accuracy)

{0: 0, 1: 1}
Training Accuracy:  70.67333333333333
{0: 1, 1: 1}
Testing Accuracy:  49.61
